In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

# set a random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


## On récupère la donnée

In [20]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_14153/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [21]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [22]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [23]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [24]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On merge les customer et leur localisation

In [25]:
# merge train_customers and train_locations
train = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id')

print('train shape: ', train.shape)
train.head()

train shape:  (58213, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,0,Work,-96.4,-67.2
1,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,1,Home,-96.6,-67.2
2,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,2,NaN,-0.1,-78.6
3,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,0,Home,-0.2,-78.6
4,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,1,Home,0.2,-78.6


In [26]:
# merge test_customers and test_locations
test = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id')

print('test shape: ', test.shape)
test.head()

test shape:  (16333, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2
1,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6
2,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7
3,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6
4,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1


### On arrange la donnée pour notre réseau

on créé une matrice, avec en ligne les clients et en colonne les vendeurs, on met a 1 les couples (client, vendeur) apparaissants dans orders

In [27]:
# we create a matrix with all the possible combinations of CID as lines and VENDOR as columns
# we fill the matrix with 0 if the combination is not in check_orders
# we fill the matrix with 1 if the combination is in check_orders

# we create a list of all the CID
CID_list = train_customers['akeed_customer_id'].unique()
print('CID_list shape: ', CID_list.shape)

# we create a list of all the VENDOR
VENDOR_list = vendors['id'].unique()
print('VENDOR_list shape: ', VENDOR_list.shape)

# we create a matrix with all the possible combinations of CID as lines and VENDOR as columns
matrix = np.zeros((len(CID_list), len(VENDOR_list)))

# we fill the matrix with 1 if the combination is in orders
for i in orders.index:
    CID = orders['customer_id'][i]
    VENDOR = orders['vendor_id'][i]
    if CID in CID_list and VENDOR in VENDOR_list:
        matrix[np.where(CID_list == CID)[0][0], np.where(VENDOR_list == VENDOR)[0][0]] = 1

print('matrix shape: ', matrix.shape)

# we create a dataframe with the matrix
df = pd.DataFrame(matrix, index=CID_list, columns=VENDOR_list)
df.head()


CID_list shape:  (34523,)
VENDOR_list shape:  (100,)
matrix shape:  (34523, 100)


,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
TCHWPBT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZGFSYCZ,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
S2ALZFL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
952DBJQ,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1IX6FXS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Nettoyage de la donnée

In [28]:
# train_customers cleaning
train_customers = train_customers[['']]

KeyError: "None of [Index([''], dtype='object')] are in the [columns]"

In [ ]:
# keep only the columns we need
df_input = df_input[[#'gender', 
                     #'dob',
       #'status_x', 'verified_x', 'language_x',
        'latitude', 'longitude',
       #'vendor_category_en',
       'vendor_category_id', 
       'delivery_charge',
       'serving_distance',
       'is_open',
       'prepration_time', 
       #'commission', 
       'is_akeed_delivering',
       'discount_percentage', 
       #'status_y', 'verified_y', 
       'rank', 
       #'language_y',
       'vendor_rating',
       # 'primary_tags',
       #'open_close_flags', 
       #'one_click_vendor',
       #'country_id', 'city_id', 
       #'device_type',
       #'display_orders',
       # 'vendor_tag', 'vendor_tag_name',
       ]]

# print all the columns with their number of missing values
print('number of missing values per column:')
display(df_input.isnull().sum())

# replace the missing values in gender with "None"
#df_input['gender'] = df_input['gender'].fillna('None')

# replace the missing values in dob with the mean
#df_input['dob'] = df_input['dob'].fillna(df_input['dob'].mean())

# replace the missing values in language_x with "None"
#df_input['language_x'] = df_input['language_x'].fillna('None')

# replace the missing values in commission with the mean
#df_input['commission'] = df_input['commission'].fillna(df_input['commission'].mean())

# replace the missing values in Language_y with "None"
#df_input['language_y'] = df_input['language_y'].fillna('None')

# replace primary_tags with "None"
#df_input['primary_tags'] = df_input['primary_tags'].fillna('None')

# replace vendor_tag_name with "None"
#df_input['vendor_tag_name'] = df_input['vendor_tag_name'].fillna('None')

# replace vendor_tag with "None"
#df_input['vendor_tag'] = df_input['vendor_tag'].fillna('None')


number of missing values per column:


latitude               0
longitude              0
vendor_category_id     0
delivery_charge        0
serving_distance       0
is_open                0
prepration_time        0
is_akeed_delivering    0
discount_percentage    0
rank                   0
vendor_rating          0
dtype: int64

In [ ]:
# check that the number of rows is the same as in all_pairs
print('number of rows in df_input: ', df_input.shape[0])
print('number of rows in all_pairs: ', all_pairs.shape[0])

number of rows in df_input:  160284
number of rows in all_pairs:  160284


In [ ]:
# print the types of the columns
print('df_input types: ', df_input.dtypes)

# label encoding
from sklearn.preprocessing import LabelEncoder

# create a LabelEncoder object
le = LabelEncoder()

#df_input['status_x'] = le.fit_transform(df_input['status_x'])
#df_input['verified_x'] = le.fit_transform(df_input['verified_x'])
#df_input['language_x'] = le.fit_transform(df_input['language_x'])
#df_input['gender'] = le.fit_transform(df_input['gender'])
#df_input['vendor_category_en'] = le.fit_transform(df_input['vendor_category_en'])
df_input['is_akeed_delivering'] = le.fit_transform(df_input['is_akeed_delivering'])
#df_input['language_y'] = le.fit_transform(df_input['language_y'])
#df_input['primary_tags'] = le.fit_transform(df_input['primary_tags'])
#df_input['one_click_vendor'] = le.fit_transform(df_input['one_click_vendor'])

df_input types:  latitude               float64
longitude              float64
vendor_category_id     float64
delivery_charge        float64
serving_distance       float64
is_open                float64
prepration_time          int64
is_akeed_delivering     object
discount_percentage    float64
rank                     int64
vendor_rating          float64
dtype: object


In [ ]:
# print clean data
print('df_input shape: ', df_input.shape)
df_input.head()

df_input shape:  (160284, 11)


,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,is_akeed_delivering,discount_percentage,rank,vendor_rating
0,-1.0,8.1e-02,2.0,0.0,15.0,1.0,12,0,0.0,11,4.5
1,-1.0,8.1e-02,2.0,0.0,15.0,1.0,12,0,0.0,11,4.5
2,-1.0,8.1e-02,2.0,0.0,15.0,1.0,12,0,0.0,11,4.5
3,-1.0,8.1e-02,2.0,0.0,15.0,1.0,12,0,0.0,11,4.5
4,-1.0,8.1e-02,2.0,0.0,15.0,1.0,12,0,0.0,11,4.5


### Création des data splits

In [ ]:
# data split
train, test = train_test_split(df_input, test_size=0.2, random_state=42, shuffle=True)
print('test shape: ', test.shape)
print('train shape: ', train.shape)

y_train = all_pairs['PREDICTION']
y_train, y_test = train_test_split(y_train, test_size=0.2, random_state=42, shuffle=True)

test shape:  (32057, 11)
train shape:  (128227, 11)


### Creation du modèle

In [ ]:
# creation of an ai model
# it takes as input a dataframe with the columns as in df_input
# it returns a dataframe with the columns as in all_pairs and a column PREDICTION with the prediction

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

model = keras.Sequential()
model.add(Input(shape=(df_input.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 256)               3072      
                                                                 
 dense_63 (Dense)            (None, 128)               32896     
                                                                 
 dense_64 (Dense)            (None, 64)                8256      
                                                                 
 dense_65 (Dense)            (None, 1)                 65        
                                                                 
Total params: 44,289
Trainable params: 44,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit the model
model.fit(train, y_train, epochs=10, batch_size=32, validation_data=(test, y_test))


Epoch 1/10
4008/4008 [==============================] - 4s 792us/step - loss: 0.2456 - accuracy: 0.5572 - val_loss: 0.2421 - val_accuracy: 0.5724
Epoch 2/10
4008/4008 [==============================] - 3s 773us/step - loss: 0.2403 - accuracy: 0.5755 - val_loss: 0.2393 - val_accuracy: 0.5743
Epoch 3/10
4008/4008 [==============================] - 3s 770us/step - loss: 0.2387 - accuracy: 0.5875 - val_loss: 0.2370 - val_accuracy: 0.5917
Epoch 4/10
4008/4008 [==============================] - 3s 781us/step - loss: 0.2361 - accuracy: 0.5935 - val_loss: 0.2373 - val_accuracy: 0.5932
Epoch 5/10
4008/4008 [==============================] - 3s 782us/step - loss: 0.2352 - accuracy: 0.5974 - val_loss: 0.2342 - val_accuracy: 0.6032
Epoch 6/10
4008/4008 [==============================] - 3s 782us/step - loss: 0.2347 - accuracy: 0.6003 - val_loss: 0.2333 - val_accuracy: 0.6068
Epoch 7/10
4008/4008 [==============================] - 3s 779us/step - loss: 0.2334 - accuracy: 0.6036 - val_loss: 0.2328 -